<a href="https://colab.research.google.com/github/royam0820/crewai/blob/main/L4_tools_customer_outreach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tools for a Customer Outreach Campaign

This Jupyter notebook outlines a **customer outreach campaign using AI agents**. It leverages the crewai library to create and manage AI **agents responsible for identifying and nurturing high-value leads through personalized communications**.

The workflow involves:
- profiling potential leads and
- crafting targeted outreach campaigns.


In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

## Library Installation

In [ ]:
!pip install crewai==0.28.8
!pip install crewai_tools==0.1.6
!pip install langchain_community==0.0.29 # this is formatted as code
!pip install utils

## Warnings Suppression

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

## Importing Required Classes

In [ ]:
from crewai import Agent, Task, Crew
import os
from google.colab import userdata

## Setting API Keys and LLM

In [ ]:
openai_api_key = userdata.get("OPENAI_API_KEY")
serper_api_key = userdata.get("SERPER_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

## Defining Agents

### Sales Representative
The Sales Representative agent is responsible for identifying potential leads.

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

### Lead Sales Representative

The Lead Sales Representative agent nurtures leads through personalized communications.

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Defining Tools


In [ ]:
!pip install crewai_tools

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [ ]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Defining Tasks

- The Lead Profiling Task is using crewAI Tools.

### Lead Profiling Task

This task involves the Sales Representative agent profiling potential leads.

The Lead Profiling Task is using crewAI Tools.

In [ ]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

### Personalized Outreach Task

This task involves the Lead Sales Representative agent crafting personalized outreach campaigns.

The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).


In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Assembling the Crew and Executing the Workflow

In [ ]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=True
)

## Inputs and Execution

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To begin compiling a detailed profile on DeepLearningAI, I should start by searching the internet to gather preliminary information about the company, its key decision-makers, recent developments, and its potential needs.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: Deeplearning.ai - Crunchbase Company Profile & Funding
Li

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

Subject: Empowering DeepLearningAI's Mission through Innovative AI Solutions

Dear Andrew,

I hope this message finds you well. I'm reaching out to celebrate DeepLearningAI's recent successes in launching the new Machine Learning Specialization and updated courses in collaboration with tech leaders like Google and AWS. Your commitment to democratizing AI education aligns perfectly with our mission at CrewAI.

At CrewAI, we specialize in developing AI-driven solutions that enhance online learning experiences. Our tools can provide deep insights into learner behaviors, enabling you to offer more personalized and effective educational programs. Moreover, our collaboration could further innovate your platforms, making AI education even more accessible and impactful across the globe.

I would love the opportunity to discuss how our solutions could support DeepLearningAI's goals and contribute to the future of AI education. Are you available for a brief call next week to explore this potential partnership?

Looking forward to your thoughts.

Best regards,
[Your Name]
Lead Sales Representative, CrewAI

# Conclusion
This notebook demonstrates how to automate a customer outreach campaign using a multi-agent system, ensuring personalized and strategic communications to high-value leads. ​